In [ ]:
# TASK = AE, AP

TASK = 'AP'
print(TASK)

reload = True

In [ ]:
!pip install lightning
!pip install fasttext
!pip install comet_ml

In [ ]:
import numpy as np
import pickle
import pandas as pd
import json
import os
import ast
import math
import re
import copy

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.autograd import Variable

CUDA_LAUNCH_BLOCKING=1

In [ ]:
# metric
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
import comet_ml

comet_ml.init(project_name="MASAD")
#RDaPLQIi9AFz0Eaeu2PEQg0X

In [ ]:
import lightning as L
from pytorch_lightning.loggers import CometLogger
import pytorch_lightning as pl

comet_logger = CometLogger()

In [ ]:
from gensim.models import FastText
from PIL import Image

In [ ]:
aspect = {'spider': 0,   'road': 1,    'dance': 2,     'milk': 3,       'horse': 4, 
          'cloud': 5,    'glass': 6,   'insect': 7,    'flower': 8,     'eye': 9, 
          'apple': 10,   'dog': 11,    'river': 12,    'leave': 13,     'church': 14, 
          'winter': 15,  'bird': 16,   'factory': 17,  'night': 18,     'car': 19, 
          'coast': 20,   'teen': 21,   'face': 22,     'house': 23,     'tree': 24, 
          'boat': 25,    'castle': 26, 'mushroom': 27, 'bady': 28,      'beach': 29, 
          'fish': 30,    'socks': 31,  'street': 32,   'girl': 33,      'chair': 34, 
          'feet': 35,    'autumn': 36, 'food': 37,     'forest': 38,    'cat': 39, 
          'rose': 40,    'band': 41,   'train': 42,    'chocolate': 43, 'grandfather': 44, 
          'boy': 45,     'plant': 46,  'ocean': 47,    'waterfall': 48, 'toy': 49, 
          'cupcake': 50, 'hat': 51,    'tshirt': 52,   'meat': 53,      'graden': 54, 
          'market': 55,  'cake': 56}

polarity = {
    'positive': 0,
    'negative': 1
}

In [ ]:
config_img = {
    'image_size': 224,
    'patch_size': 16,
    'num_channels': 3,
    'num_heads': 12,
    'num_layers': 12,
    'input_dim': 2048,
    'hidden_dim': 768,
    'output_dim': 1000,
}

config_text = {
    'input_dim': 300,
    'hidden_dim':  300, 
    'embedding': 300,
    'num_layers': 5,
    'output_dim': 1000,
    'num_channels': 3,
    'num_heads': 12,#embedding phai chia het cho num_heads
}

BATCH_SIZE=32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def evalution(labels, predicts):
    accuracy = accuracy_score(labels, predicts)
    f1 = f1_score(labels, predicts, average='weighted', zero_division = 0)
    recall = recall_score(labels, predicts, average='weighted', zero_division = 0)
    precision = precision_score(labels, predicts, average='weighted', zero_division = 0)

    accuracy = round(accuracy * 100, 4)
    f1 = round(f1 * 100, 4)
    recall = round(recall * 100, 4)
    precision = round(precision * 100, 4)

    return accuracy, f1, precision, recall

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    return data

data_raw_train = read_file('/kaggle/input/masadtext/train.json')
data_raw_test = read_file('/kaggle/input/masadtext/test.json')

In [ ]:
output_file = '/kaggle/working/text.model'

if os.path.isfile(output_file):
    print(f"File {output_file} already exists. Aborting.")
else:
    sentences = []
    
    all_data = data_raw_train
    for d in all_data:
        sentences.append(re.sub(r'\([^)]*\)', '', ' '.join(d['token'])).split())

    all_data = data_raw_test
    for d in all_data:
        sentences.append(re.sub(r'\([^)]*\)', '', ' '.join(d['token'])).split())

    model_ted = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4, sg=1)
    
    word_vectors = model_ted.wv

    model_ted.save(output_file)
    
text_model = FastText.load("text.model")
text_model = text_model.wv

print(len(text_model.index_to_key))

In [ ]:
word_dict = {'<unk>': 0 }
for word in text_model.index_to_key:
    word_dict[word] = len(word_dict)

embedding = []
embedding.append(np.zeros(300))

for key in word_dict.keys():
    embedding.append(text_model[key])
    
embedding = np.array(embedding)
embedding = torch.Tensor(embedding).to(device)

print(word_dict)

In [ ]:
output_file = '/kaggle/working/aspect.model'

if os.path.isfile(output_file):
    print(f"File {output_file} already exists. Aborting.")
else:
    sentences = []

    for key in aspect.keys():
        sentences.append(key)

    model_ted = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4, sg=1)

    word_vectors = model_ted.wv

    model_ted.save(output_file)
    
aspect_model = FastText.load("aspect.model")

aspect_weight = []
for key in aspect.keys():
    x = aspect_model.wv[key]
    x = np.array([x] * 300) 
    x = (x + x.T) / 2
    
    aspect_weight.append(x)
    
aspect_weight = torch.Tensor(aspect_weight).to(device)

In [ ]:
p = {
    '<unk>': 0
}
all_data = data_raw_train
for d in all_data:
    for i in d['pos']:
        if i not in p:
            p[i] = len(p)

all_data = data_raw_test
for d in all_data:
    for i in d['pos']:
        if i not in p:
            p[i] = len(p)

print(p)

num_pos = len(p)

In [ ]:
output_file = '/kaggle/working/tag.model'

if os.path.isfile(output_file):
    print(f"File {output_file} already exists. Aborting.")
else:
    sentences = []
    
    all_data = data_raw_train
    for d in all_data:
        for i in d['tag']:
            sentences.append(i)

    all_data = data_raw_test
    for d in all_data:
        for i in d['tag']:
            sentences.append(i)

    model_ted = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4, sg=1)

    word_vectors = model_ted.wv

    model_ted.save(output_file)

tag_model = FastText.load("tag.model")
tag_model = tag_model.wv

In [ ]:
class datasets(Dataset):
    def __init__(self, data):
        self.data = data
        self.transforms = transforms.Compose([
            transforms.Resize((config_img['image_size'], config_img['image_size'])),
            transforms.ToTensor()
        ])
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # text
        token = []
        n = self.data[idx]['token']
        for i in range(config_text['embedding']):
            if (i < len(n) and n[i] in word_dict):
                token.append(word_dict[n[i]])
            else:
                token.append(0)
        token = torch.Tensor(token).to(device)
        
        #aspect
        asp = self.data[idx]['aspect']
        asp_ids = aspect[asp]
        
        if(TASK == 'AP'):
            label = polarity[self.data[idx]['polarity']]
        else:
            label = aspect[self.data[idx]['aspect']]
        
        # image
        img_path = self.data[idx]['imgpath']
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            image = Image.new("RGB", (config_img['image_size'], config_img['image_size']), (0, 0, 0))
            
        image = self.transforms(image)
        
        # tag
        list_tag =  self.data[idx]['tag']
        vector_tag = []
        
        for i in range(50):
            if(i < len(list_tag)):
                vector_tag.append(tag_model[list_tag[i]])
            else:
                vector_tag.append(np.zeros(300))
        
        vector_tag = torch.tensor(vector_tag)
        
        #onehot
        vector_onehot = torch.zeros(len(aspect))
        vector_onehot[aspect[asp]] = 1
        
        #short
        mask_0 = [[-99999] * config_text['embedding'] for _ in range(config_text['embedding'])]
        mask_1 = [[-99999] * config_text['embedding'] for _ in range(config_text['embedding'])]
        mask_2 = [[-99999] * config_text['embedding'] for _ in range(config_text['embedding'])]
        mask_3 = [[-99999] * config_text['embedding'] for _ in range(config_text['embedding'])]
        mask_4 = [[-99999] * config_text['embedding'] for _ in range(config_text['embedding'])]
        short_length = min(len(self.data[idx]['short']), config_text['embedding'])
        for i in range(short_length):
            for j in range(short_length):
                mask_0[i][j] = 0
                if self.data[idx]['short'][i][j] == 1:
                    mask_1[i][j] = 0 
                    mask_2[i][j] = 0
                    mask_3[i][j] = 0
                    mask_4[i][j] = 0
                elif self.data[idx]['short'][i][j] == 2:
                    mask_2[i][j] = 0
                    mask_3[i][j] = 0
                    mask_4[i][j] = 0
                elif self.data[idx]['short'][i][j] == 3:
                    mask_3[i][j] = 0
                    mask_4[i][j] = 0
                elif self.data[idx]['short'][i][j] == 4:
                    mask_4[i][j] = 0

        for i in range(short_length):
            mask_1[i][i] = 0 
            mask_2[i][i] = 0
            mask_3[i][i] = 0
            mask_4[i][i] = 0
        
        short_mask = []
        short_mask.append(mask_0)
        short_mask.append(mask_1)
        short_mask.append(mask_2)
        short_mask.append(mask_3)
        short_mask.append(mask_4)
        
        short_mask = torch.Tensor(short_mask).to(device)
            
#         pos 
#         pos = []
#         n = self.data[idx]['pos']
#         for i in range(config_text['embedding']):
#             if (i < len(n)):
#                 pos.append(p[n[i]])
#             else:
#                 pos.append(0)
                
#         pos = torch.Tensor(pos).to(device)
        
        output = {
            "token": token,
            'aspect': asp,
            'img': image,
            'label': label,
            'asp_ids': asp_ids,
            'onehot': vector_onehot,
            'tag': vector_tag,
            'short': short_mask,
            'pos': 1
        }
        
        return output

reload = True

In [ ]:
print(reload)

def load_data(file_path, data_raw, reload = True):
    data = []
    
    if reload:
        data = datasets(data_raw)
        torch.save(data, file_path)
        return data
    else:
        data = torch.load(file_path, map_location=device)  
        return data 

data_train = load_data('/kaggle/working/data_train.pth', data_raw_train, reload)
data_test = load_data('/kaggle/working/data_test.pth', data_raw_test, reload)

print(len(data_train))
print(len(data_test))
    
reload = False

In [ ]:
train_dataloader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(data_test, batch_size=BATCH_SIZE)

In [ ]:
def process_args(batch):
    img = batch['img']
    token = batch['token']
    short = batch['short']
    labels = batch['label']
#     asp_ids = batch['asp_ids']
    ls_tag = batch['tag']
    onehot = batch['onehot']

    img = img.float().to(device) 
    ls_tag = ls_tag.float().to(device)
    token = token.long().to(device)
    labels = labels.long().to(device)
    onehot = onehot.float().to(device)
    short = short.float().to(device)

    return img, ls_tag, token, short, onehot, labels

In [ ]:
def rnn_zero_state(batch_size, hidden_dim, num_layers, bidirectional=True): 
    total_layers = num_layers * 2 if bidirectional else num_layers
    state_shape = (total_layers, batch_size, hidden_dim)   
    h0 = c0 = Variable(torch.zeros(*state_shape), requires_grad=False)
    return h0.to(device), c0.to(device)

def clones(module, N): # Tạo N bản sao của một module.
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def self_attention(query):
    d_k = query.size(-1) 
    scores = torch.matmul(query, query.transpose(-2, -1))
    scores = scores / math.sqrt(d_k) 
    return scores

def aspect_attention(batch, weight_m, key, bias_m):
    p=weight_m.size(0)
    mx=weight_m.size(1)
    weight_m=weight_m.unsqueeze(0).expand(batch, p, mx, mx).to(device)
    
    batch, x, y, z = key.shape
    aspect = torch.ones((batch, x, y, weight_m.size()[2])).to(device)
    H_W = torch.matmul(aspect, weight_m).to(device)
    
    aspect_scores = torch.matmul(H_W, key.transpose(-2, -1)).to(device)
    aspect_scores = torch.add(aspect_scores, bias_m).to(device)
    aspect_scores = torch.tanh(aspect_scores).to(device)
    
    return aspect_scores

def syntax_mask(scores, short):
    scores = torch.add(scores, short)
    scores = F.softmax(scores, dim=-1)
    return scores

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.h = 5
        self.hidden_size = config_text['hidden_dim']
        self.d_k = self.hidden_size // self.h
        
        self.linears = clones(nn.Linear(self.hidden_size, self.hidden_size), 2)
        self.weight_m = nn.Parameter(torch.Tensor(self.h, self.hidden_size // self.h, self.hidden_size // self.h))
        self.bias_m = nn.Parameter(torch.Tensor(1))
        
    def forward(self, query, short):
        batch = query.size(0)
        key = query
        query, key = [l(x).view(batch, -1, self.h, self.d_k).transpose(1, 2)  
                             for l, x in zip(self.linears, (query, key))]  
        scores = self_attention(query)
        
        if 'AP' in TASK:
            scores_aspect = aspect_attention(batch, self.weight_m, key, self.bias_m)
            scores = torch.add(scores, scores_aspect)
        
        scores = syntax_mask(scores, short)
        
        return scores

class GCN(nn.Module):
    def __init__(self, layer = 3, output_dim = 1000):
        super(GCN, self).__init__()
        self.layer = layer
        self.attention_heads = 5
        self.hidden_size = config_text['hidden_dim']
        self.embedding = config_text['embedding']
        
        self.W = nn.Linear(self.embedding, self.embedding) 
        
        self.gcn_drop = nn.Dropout(0.1)
        self.fc1 = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size * self.hidden_size, self.hidden_size * 10)
        self.fc3 = nn.Linear(self.hidden_size * 10, output_dim)
        
    def forward(self, inp, weight_adj): 
        out = inp
        batch = inp.shape[0]
        for i in range(self.layer):
            out = out.unsqueeze(1).expand(batch, self.attention_heads, self.embedding, self.hidden_size * 2)
            out = torch.matmul(weight_adj, out).mean(dim=1)
            out = F.selu(out)
            out = self.gcn_drop(out) if i < self.layer - 1 else out 
        
        out=self.fc1(out)
        out = out.reshape(out.shape[0],  -1)
        out=self.fc2(out)
        out=self.fc3(out)
        out=F.relu(out)
        return out

class ModelText(nn.Module):
    def __init__(self):
        super(ModelText, self).__init__()
        self.hidden_size = config_text['hidden_dim']
        self.num_layers = config_text['num_layers']
        self.input_size = config_text['input_dim']
        self.embedding = config_text['embedding']
        self.h = 5
        self.embs = nn.Embedding.from_pretrained(torch.tensor(embedding, dtype=torch.float), freeze=True)
        self.pos_embs = nn.Embedding(num_pos, self.embedding, dtype=torch.float,padding_idx=0)
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True, bidirectional=True)
        self.attention = Attention().to(device)
        self.gcn = GCN().to(device)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(1000, len(aspect))
            
    def encode_with_rnn(self, rnn_inputs):
        batch_size = len(rnn_inputs)
        h0, c0 = rnn_zero_state(batch_size, self.hidden_size, self.num_layers) 
        rnn_outputs, (ht, ct) = self.lstm(rnn_inputs, (h0, c0))
        return rnn_outputs
    
    def forward(self, x, short):
        x = self.embs(x)
        out = self.encode_with_rnn(x)
        gcn_input = self.attention(x, short)
        out = self.gcn(out, gcn_input)
        out = self.fc(out)
        return out.squeeze(1)
    
model_text = ModelText().to(device)
print(model_text)

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, 1000)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet152():
    model = ResNet(Bottleneck, [3, 4, 6, 3]).to(device)
    model.load_state_dict(torch.load('/kaggle/input/resnet-pretrain/resnet50-19c8e357.pth', map_location='cpu'))
    return model.to(device)

class myResnet(nn.Module):
    def __init__(self):
        super(myResnet, self).__init__()
        self.resnet = resnet152().to(device)
        
        self.noun_linear=nn.Linear(300, config_img['hidden_dim'])
        self.multi_linear=nn.Linear(config_img['input_dim'], config_img['hidden_dim'])
        self.att_linear=nn.Linear(config_img['hidden_dim']*2, 1)
        self.linear=nn.Linear(config_img['hidden_dim']*2, 1)
        self.alpha_linear1=nn.Linear(config_img['input_dim'], config_img['hidden_dim'])
        self.alpha_linear2=nn.Linear(config_img['input_dim'], config_img['hidden_dim'])
        self.fc = nn.Linear(300, config_img['input_dim'])
        self.fc1 = nn.Linear(1000 + config_img['input_dim'] * 7 * 7, len(aspect))
        
        self.tag_att = self.noun_attention
    
    def noun_attention(self, encoder_outputs, noun_embed):
        encoder_outputs = encoder_outputs.float().to(device)
        noun_embed = noun_embed.float().to(device)
        multi_features_rep = encoder_outputs.unsqueeze(2).repeat(1, 1, noun_embed.shape[1], 1)
        noun_features_rep = noun_embed.unsqueeze(1).repeat(1, encoder_outputs.shape[1], 1, 1)
        noun_features_rep = self.noun_linear(noun_features_rep)
        multi_features_rep = self.multi_linear(multi_features_rep)
        concat_features = torch.tanh(torch.cat([noun_features_rep, multi_features_rep], dim=-1))
        att = torch.softmax(self.att_linear(concat_features).squeeze(-1), dim=-1)
        att_features = self.fc(torch.matmul(att, noun_embed))
        alpha = torch.cat([self.alpha_linear1(encoder_outputs), self.alpha_linear2(att_features)], dim=-1)
        alpha = torch.sigmoid(self.linear(alpha))
        alpha = alpha.repeat(1, 1, 1)

        encoder_outputs = torch.mul(1 - alpha, encoder_outputs)
        encoder_outputs += torch.mul(alpha, att_features)
        return encoder_outputs

    def forward(self, x, list_tag = None, att_size=7):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        fc = x.mean(3).mean(2)
        att = F.adaptive_avg_pool2d(x,[att_size,att_size])
        att = att.view(-1, 2048, 49).permute(0, 2, 1)
        
        list_tag = list_tag.double()
        
        abc = self.tag_att(encoder_outputs=att, noun_embed=list_tag)

        x = self.resnet.avgpool(x)
        x = x.view(x.size(0), -1)
        abc = abc.reshape(abc.size(0), -1)
        
        x = self.resnet.fc(x)
        
        x = torch.cat((x, abc), dim = 1)
        x = self.fc1(x)

        return x
    
model_img = myResnet().to(device)
print(model_img)

In [ ]:
class model_full(pl.LightningModule):
    def __init__(self):
        super(model_full, self).__init__()
        
        self.text = model_text
        self.img = model_img
        
        if 'AP' in TASK:
            self.fc = nn.Sequential(
                nn.Linear((3 * len(aspect)), len(aspect)),
                nn.Linear((len(aspect)), 2)
            )
            
        if 'AE' in TASK:
            self.fc = nn.Linear((2 * len(aspect)), len(aspect))
            
        self.criterion = nn.CrossEntropyLoss()
        
        self.test_preds = []
        self.test_labels = []
        self.matrix = []
        
        #comel
        self.save_hyperparameters()
        
    def forward(self, img, list_tag, text,short, onehot):
        text = self.text(text, short)
        img = self.img(img, list_tag)
        
        outputs = torch.cat((text, img), dim=1)
        
        if 'AP' in TASK:
            outputs = torch.cat((outputs, onehot), dim=1)
        
        outputs = self.fc(outputs)
        
        return outputs
#      img, ls_tag, token, short, onehot, labels
    
    def training_step(self, batch, batch_idx):
        img, ls_tag, token, short, onehot, labels = process_args(batch)
        outputs = self(img, ls_tag, token, short, onehot)
        loss = self.criterion(outputs, labels)
        self.log('loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        img, ls_tag, token, short, onehot, labels = process_args(batch)
        outputs = self(img, ls_tag, token, short, onehot)
        
        predicts = torch.argmax(outputs, dim=1)
        labels = labels.cpu().numpy()
        predicts = predicts.cpu().numpy()
        
        self.test_preds.extend(predicts)
        self.test_labels.extend(labels)
        
        acc, f1, prec, rec = evalution(labels, predicts)
        
        return {'val_acc': acc, 'val_f1': f1, 'val_prec': prec, 'val_rec': rec}

    def test_step(self, batch, batch_idx):
        img, ls_tag, token, short, onehot, labels = process_args(batch)
        outputs = self(img, ls_tag, token, short, onehot)
        
        predicts = torch.argmax(outputs, dim=1)
        labels = labels.cpu().numpy()
        predicts = predicts.cpu().numpy()
        
        self.test_preds.extend(predicts)
        self.test_labels.extend(labels)
        
        acc, f1, prec, rec = evalution(labels, predicts)
        
        return {'test_acc': acc, 'test_f1': f1, 'test_prec': prec, 'test_rec': rec}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.00001)
        self.logger.experiment.log_metric('lr', optimizer.param_groups[0]['lr'])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_acc'}
    
    def on_validation_epoch_end(self):
        acc, f1, prec, rec = evalution(self.test_labels, self.test_preds)
        self.write(inp = {'val_acc': acc, 'val_f1': f1, 'val_prec': prec, 'val_rec': rec})
        self.test_labels = []
        self.test_preds = []
        
    def on_test_epoch_end(self):
        acc, f1, prec, rec = evalution(self.test_labels, self.test_preds)
        self.write(inp = {'val_acc': acc, 'val_f1': f1, 'val_prec': prec, 'val_rec': rec})
        self.test_labels = []
        self.test_preds = []
    
    def write(self, inp):
        for key in inp.keys():
            self.log(key, inp[key], on_epoch=True, prog_bar=True, logger=True)
    
    def getDomain(self):
        return self.matrix
    
    def getLabelTest(self):
        return self.test_labels
    
    def getLabelPred(self):
        return self.test_preds


model = model_full().to(device)
print(model)

In [ ]:
NUM_EPOCHS = 2
comet_logger.log_hyperparams({"batch_size": BATCH_SIZE})

accelerator = "cuda" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, accelerator=accelerator, logger=comet_logger)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
trainer.test(model, dataloaders=test_dataloader)

In [ ]:
test_preds = model.getLabelPred()
test_labels = model.getLabelTest()
print("Test Classification Report:\n", classification_report(test_labels, test_preds))

In [ ]:
conf_matrix = confusion_matrix(test_labels, test_preds)

plt.figure(figsize=(8, 6))
ax = sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True)

tick_labels = [] 

if TASK == 'AE':
    for key in aspect.keys():
        tick_labels.append(key)
else:
    for key in polarity.keys():
        tick_labels.append(key)
    
    
ax.set_xticklabels(tick_labels, rotation=0)
ax.set_yticklabels(tick_labels, rotation=0)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
food = {
    "apple": 0, 
    "food": 0, 
    "milk": 0, 
    "mushroom": 0,
    "chocolate":0,
    "meat":0,
    "cake":0
} # 6

goods = {
    "boat": 0, 
    "car": 0, 
    "chair": 0, 
    "glass": 0, 
    "socks": 0, 
    "train": 0,
    "cupcake":0,
    "hat":0,
    "toy":0,
    "tshirt":0
}#10

buildings = {
    "castle": 0,
    "church": 0,
    "factory": 0,
    "house": 0,
    "road": 0,
    "market":0
}#6

animal = {
    "bird": 0,
    "cat" : 0,
    "dog": 0,
    "fish": 0,
    "horse": 0,
    "insect": 0,
    "spider": 0
}#7

human = {
    "bady": 0, # masad viết sai chính tả 'baby'
    "band": 0,
    "dance": 0, 
    "eye": 0, 
    "face": 0, 
    "feet": 0, 
    "girl": 0, 
    "teen": 0,
    "boy":0,
    "grandfather":0
}#10

plant = {
    "flower": 0,
    "rose": 0,
    "tree": 0, 
    "leave": 0,
    "plant":0
}#5

scenery = {
    "autumn": 0,
    "beach": 0,
    "cloud": 0,
    "coast": 0,
    "night": 0,
    "river": 0,
    "street": 0,
    "winter": 0,
    "graden":0, #garden is wrong write
    "ocean":0,
    "waterfall":0,
    "forest":0
} #12

In [ ]:
matrix =  model.getDomain()

In [ ]:
food_true = 0
total_food = 0

goods_true = 0
total_goods = 0

buildings_true = 0
total_builddings = 0

animal_true = 0
total_animal = 0

human_true = 0
total_human = 0

plant_true = 0
total_plant = 0

scenery_true = 0
total_scenery = 0

for asp, predict in matrix:
    if asp in food:
        food_true += (predict == True)
        total_food += 1
        
    elif asp in goods:
        goods_true += (predict == True)
        total_goods += 1
        
    elif asp in buildings:
        buildings_true += (predict == True)
        total_builddings += 1
        
    elif asp in animal:
        animal_true += (predict == True)
        total_animal += 1
        
    elif asp in human:
        human_true += (predict == True)
        total_human += 1
        
    elif asp in plant:
        plant_true += (predict == True)
        total_plant += 1
    elif asp in scenery:
        scenery_true += (predict == True)
        total_scenery += 1
    else:
        print(asp, predict)
        
x = 0
        
accuracy = 100 * scenery_true / (total_scenery + 1e-10)
print(f'Accuracy on test domain scenery: {accuracy}%')

x += accuracy

accuracy = 100 * food_true / (total_food+ 1e-10)
print(f'Accuracy on test domain food: {accuracy}%')

x += accuracy

accuracy = 100 * goods_true / (total_goods+ 1e-10)
print(f'Accuracy on test domain goods: {accuracy}%')

x += accuracy

accuracy = 100 * buildings_true / (total_builddings+ 1e-10)
print(f'Accuracy on test domain buildings: {accuracy}%')

x += accuracy

accuracy = 100 * animal_true / (total_animal+ 1e-10)
print(f'Accuracy on test domain animal: {accuracy}%')

x += accuracy

accuracy = 100 * human_true / (total_human+ 1e-10)
print(f'Accuracy on test domain human: {accuracy}%')

x += accuracy

accuracy = 100 * plant_true / (total_plant+ 1e-10)
print(f'Accuracy on test domain plant: {accuracy}%')

x += accuracy

print('-------------------')
print(x/7)

print(scenery_true + food_true + goods_true + buildings_true + animal_true + human_true + plant_true)
print(total_scenery + total_food + total_goods + total_builddings + total_animal + total_human + total_plant)

print('-------------------')
x = scenery_true + food_true + goods_true + buildings_true + animal_true + human_true + plant_true
y = total_scenery + total_food + total_goods + total_builddings + total_animal + total_human + total_plant

print(x/y)

print('-------------------')
x = scenery_true/total_scenery + food_true/total_food + goods_true/total_goods + buildings_true/total_builddings + animal_true/total_animal + human_true/total_human + plant_true/total_plant
print(x/7)

print('-------------------')
print(scenery_true/total_scenery)
print(food_true/total_food)
print(goods_true/total_goods)
print(buildings_true/total_builddings)
print(animal_true/total_animal)
print(human_true/total_human)
print(plant_true/total_plant)